<a href="https://colab.research.google.com/github/gretiere545/tradunion/blob/master/corpus_glossary_builder_v2_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Création d'un glossaire alphabétique des Corpus ASAMLA

## Chargement des Corpus

In [ ]:
#!/usr/bin/env python
# -*- coding: utf8 -*-

!pip install fpdf
!pip install arabic_reshaper
# https://pyfpdfbook.wordpress.com/2015/03/22/putting-two-adjacent-multicell-blocks/
import pandas as pd
import numpy as np
from fpdf import FPDF
import arabic_reshaper
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
!export PYTHONIOENCODING=utf8
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [ ]:
uploaded_cover = files.upload()

Saving 2732x2732-stil-de-grain-yellow-solid-color-background.jpg to 2732x2732-stil-de-grain-yellow-solid-color-background.jpg


In [ ]:
# cyrillique
!apt-get update -qq
!apt-get install -y fonts-dejavu-core -qq

# amharique
!apt-get update
!apt-get install fonts-noto




In [ ]:
# arabic
!apt-get update
!apt-get install lemonada


In [3]:
# Ouverture de la Sheet Centrale (corpus_central_base)
sheet_central = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(sheet_central)
#ouverture de l'onglet Corpus dans un DF
t_corpus = wb_central.worksheet('med_vac_synthese')
data_t_corpus = t_corpus.get_all_values()
df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])
df = df_corpus[['uid','expression','rus','index']]



In [4]:
def charge_corpus_langue(df_corpus, langue):
  #ouverture de l'onglet Corpus dans un DF
  df = df_corpus[['uid','expression', langue,'index']]
  return df


In [6]:
vk_lang_dict=[
{'language':'Arabe','trigramme':'ams'},
{'language':'Turc','trigramme':'tur'},
{'language':'Russe','trigramme':'rus'},
{'language':'Ukrainien','trigramme':'ukr'},
{'language':'Roumain','trigramme':'rou'},
{'language':'Hongrois','trigramme':'hun'},
{'language':'Tigrinya','trigramme':'tig'},
{'language':'Albanais','trigramme':'alb'},
{'language':'Géorgien','trigramme':'geo'},
{'language':'Arménien','trigramme':'arm'},
{'language':'Dari','trigramme':'dar'},
{'language':'Pashto','trigramme':'pst'},
{'language':'Fârsi','trigramme':'prs'},
{'language':'Azéri','trigramme':'aze'},
{'language':'Espagnol','trigramme':'esp'},
{'language':'Amharique','trigramme':'amh'},
{'language':'Peul','trigramme':'fma'},
]

In [28]:
vk_color_theme=[
{'name':'Abstract vector geometric pattern. Symmetrical layout. Illustration eps 10.','color_1':'128, 191, 162','color_2':'137, 166, 93','color_3':'217, 184, 85','color_4':'217, 170, 85','color_5':'242, 242, 242'},
{'name':'color theme_IMG_2040','color_1':'220, 118, 70','color_2':'243, 161, 75','color_3':'147, 173, 164','color_4':'191, 219, 207','color_5':'234, 223, 201'},               
]

In [29]:
vk_color_theme[1]['name']
tuple(map(int, vk_color_theme[1]['color_1'].split(', ')))

(220, 118, 70)

In [46]:
class PDF(FPDF):
  def __init__(self):
    super().__init__()
    self.WIDTH = 210
    self.HEIGHT = 297
    self.format = 'A4'
    self.unit = 'in'
    self.set_margins(20.0, 10.0, 20.0)
    self.color_theme = vk_color_theme[0]
    self.color_1 = tuple(map(int, self.color_theme['color_1'].split(', ')))
    self.color_2 = tuple(map(int, self.color_theme['color_2'].split(', ')))   
    self.color_3 = tuple(map(int, self.color_theme['color_3'].split(', ')))
    self.color_4 = tuple(map(int, self.color_theme['color_4'].split(', ')))
    self.color_5 = tuple(map(int, self.color_theme['color_5'].split(', ')))    
    self.logo_1 = './logo_asamla.jpg'
    self.font_1 = ('Montserrat-Medium', '', 11)
    self.font_2 = ('Montserrat-Medium', '', 24)  
    self.font_cover_1 = ('BebasNeue-Regular', '', 60)  
    self.title_1 = 'Corpus Médical'
    self.title_2 = 'Vaccination'
    self.cover_img_1 = '2732x2732-stil-de-grain-yellow-solid-color-background.jpg'
    self.print_footer = False
    self.min_height_required_ln = 3.0


  def header(self):
    # Custom logo and positioning
    # Create an `assets` folder and put any wide and short image inside
    # Name the image `logo.png`
    self.image(self.logo_1, 10, 8, 33)
    self.set_text_color (*self.color_1)
    self.set_font(*self.font_1)
    self.cell(self.WIDTH - 80)
    self.cell(50, 1, self.title_1, 0, 0, 'R')
    self.ln(5)      
    self.cell(self.WIDTH - 80)
    self.set_text_color (*self.color_1)
    self.cell(50, 1, 'Français-'+self.local_language, 0, 0, 'R')      
    new_y = self.get_y()+7
    self.line(0, new_y, 210, new_y)

  def footer(self):
    # Go to 1.5 cm from bottom
    self.set_y(-15)
    # Select Arial italic 8
    self.set_font('Arial', 'I', 8)
    # Print current and total page numbers
    # Do not print footer on first page 
    if self.print_footer is True:
      self.cell(0, 10, 'Page %s' % self.page_no() + '/{nb}', 0, 0, 'C')        

  def print_page(self, images):
    # Generates the report
    self.add_page()  

  def set_langue(self, langue):
    self.local_language = langue

  def add_fonts(self):
    self.add_font('DejaVuSans', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
    self.add_font('DejaVuSans-Bold', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)    
    self.add_font('NotoSerifEthiopic', '', '/usr/share/fonts/truetype/noto/NotoSerifEthiopic-Regular.ttf', uni=True)
    self.add_font('NotoSansArabic', '', '/usr/share/fonts/truetype/noto/NotoSansArabic-Regular.ttf', uni=True) 
    self.add_font('BebasNeue-Regular','', 'BebasNeue-Regular.ttf', uni=True)   
    self.add_font('Slackey-Regular','', 'Slackey-Regular.ttf', uni=True)  
    self.add_font('Montserrat-Medium','','Montserrat-Medium.ttf', uni=True)
    self.add_font('NotoSerif-Regular','','/usr/share/fonts/truetype/noto/NotoSerif-Regular.ttf', uni=True)    
    

  def set_effective_page_width(self, value):
    self.effective_page_width = value

  def set_effective_page_height(self, value):
    self.effective_page_height = value

  def set_multi_cell_width(self, value):
    self.multi_cell_width = value

  def set_rowh(self, value):
    self.rowh = value

  def first_page(self, langue):
    self.set_font(*self.font_2)
    self.set_text_color (*self.color_1)
    # Calculate width of title and position
    w = self.get_string_width(self.title_1) + 6
    self.set_x((self.WIDTH) / 2)    
    self.set_y((self.HEIGHT) /2.5)
    self.cell(0,0, self.title_1, 0, 0, 'C',0)
    self.ln(10)
    self.set_font(*self.font_1)
    self.cell(0,0, 'Français-'+self.local_language, 0, 0, 'C')
    self.add_page()
    self.set_text_color (0,0,0)

  def cover_1(self):
    self.image(self.cover_img_1,0,0,self.WIDTH,self.HEIGHT)
    #self.set_fill_color(*self.color_3)
    #self.rect(0, 0, self.WIDTH, self.HEIGHT, style = 'F')    

    self.set_text_color(*self.color_1)
    self.set_xy(self.l_margin, 90)
    self.set_font(*self.font_cover_1)
    self.cell(0,0, self.title_1, 0, 0, 'L')
    self.ln(18)  

    self.set_font(*self.font_cover_1)
    self.set_text_color (*self.color_5)
    self.cell(0,0, self.title_2, 0, 0, 'L')
    self.ln(28)

    self.midpage = self.get_y()-14
    self.set_fill_color(*self.color_1)
    self.rect(0, self.midpage, self.WIDTH, 30, style = 'F')

    self.set_font(*self.font_2)
    self.set_text_color (*self.color_5) 
    title = 'Français-'+self.local_language
    self.cell(0,0, title.upper(), 0, 0, 'L')

    self.image(self.logo_1,self.l_margin,self.HEIGHT-self.l_margin,30)

    self.add_page()  

  def cover_2(self):
    self.set_fill_color(*self.color_1)
    self.rect(0, 0, self.WIDTH, self.HEIGHT, style = 'F')
    self.add_page()  

  def cover_3(self):
    self.set_fill_color(*self.color_1)
    self.rect(0, 0, self.WIDTH, self.HEIGHT, style = 'F')
    self.add_page()  

  def cover_4(self):
    self.image(self.cover_img_1,0,0,self.WIDTH,self.HEIGHT)
    self.set_fill_color(*self.color_1)
    self.rect(0, self.midpage, self.WIDTH, 30, style = 'F')

In [47]:
def print_expression(langue, df, pdf, ind, chosen_font_expr, chosen_font_trad):
  # impression d'une ligne expression/traduction dans la bonne fonte
  # on quitte si la traduction est absente
  if len(df[langue][ind].strip()) == 0:
    return
    
  ln = 6.5
  ybefore = pdf.get_y()
  pdf.set_font(chosen_font_expr, '', 10.0)
  pdf.set_text_color (0,0,0)
  pdf.multi_cell(pdf.multi_cell_width, pdf.rowh, df['expression'][ind])
  pdf.set_xy(pdf.effective_page_width/2 + pdf.l_margin, ybefore)
  #---- cas de l'alphabet arabe
  pdf.set_font(chosen_font_trad, '', 10.0)
  if langue=='ams':
    arabic_string = arabic_reshaper.reshape(df[langue][ind])
    arabic_string = arabic_string[::-1]
    w = pdf.get_string_width(arabic_string) + 6
    pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, arabic_string, 0, 'R')
    #pdf.cell(w, 9, arabic_string, 0, 1, 'L', 0)
  else:
    pdf.multi_cell(pdf.multi_cell_width,  pdf.rowh, df[langue][ind])

  pdf.ln(ln)


In [48]:
def create_pdf_instance(item, df):
  pdf=PDF()
  pdf.alias_nb_pages()
  pdf.set_langue (item['language'])
  pdf.add_fonts()
  pdf.format = 'A4'
  pdf.unit = 'in'
  pdf.set_margins(20.0, 10.0, 20.0)
  A4_height_inches = 11.6929
  effective_page_width = pdf.w - 2*pdf.l_margin
  effective_page_height = pdf.h - 2*pdf.b_margin
  multi_cell_width = (effective_page_width/2)-5
  pdf.set_effective_page_width(effective_page_width)
  pdf.set_effective_page_height(effective_page_height)
  pdf.set_multi_cell_width(multi_cell_width)

  rowh = 5.5
  pdf.set_rowh(rowh)
  ln = 6.5

  chosen_font_expr = 'NotoSerif-Regular'
  # cut here -------------------------------------------------------------
  if item['trigramme']=='ams':
    # font arabe
    print("Font ams")
    chosen_font_trad = 'NotoSansArabic'
  elif item['trigramme']=='tig':
    # font amharique
    print("Font tig")
    chosen_font_trad='NotoSerifEthiopic'
  elif item['trigramme']=='arm':
    # font arménien
    print("Font arm")   
    chosen_font_trad = 'DejaVuSans' 
  else:
    # font standard
    chosen_font_trad = 'NotoSerif-Regular'
    # amharique/tigrinya


  # Add new page. Without this you cannot create the document.
  pdf.add_page()
  # couverture
  pdf.cover_1()
  pdf.cover_2()
  pdf.print_footer = True
  # Remember to always put one of these at least once.
  pdf.set_font('Times','',10.0) 


  pdf.first_page(item['language'])

  pdf.ln(ln)
  idx = '' # index alpha (start)
  # pour chaque élement expression/traduction
  for ind in df.index:
    ybefore = pdf.get_y()
    if df['index'][ind] != idx:
      # changement de lettre index
      pdf.ln(ln*1.5)

      # vérification place en bas suffisante
      space_left = effective_page_height-pdf.get_y()
      # changement de page si place restante insuffisante
      if space_left < pdf.min_height_required_ln:
        pdf.add_page()
        pdf.ln(ln*2)    

      ybefore = pdf.get_y()
      # changement lettre index
      pdf.set_font(*pdf.font_2)
      pdf.set_text_color (*pdf.color_1)
      pdf.multi_cell(multi_cell_width,  rowh, df['index'][ind])
      pdf.set_xy(effective_page_width/2 + pdf.l_margin, ybefore)
      pdf.multi_cell(multi_cell_width,  rowh, " ")
      idx = df['index'][ind]
      pdf.ln(ln)

    # expression / traduction
    space_left = effective_page_height-pdf.get_y()
    #if item['trigramme']=='hun':
    #  print (df['expression'][ind], str(space_left))
    # changement de page si place restante insuffisante
    if space_left < 1:
      pdf.add_page()
      pdf.ln(ln*2)

    print_expression(item['trigramme'], df, pdf, ind, chosen_font_expr, chosen_font_trad)
    #print (df['expression'][ind], str(space_left))

  pdf.ln(ln)  
  pdf.add_page()
  pdf.print_footer = False
  pdf.cover_3()
  pdf.cover_4()
  # cut here -------------------------------------------------------------

  print ("écriture de " + item['trigramme'] + '.pdf')
  pdf.output(item['trigramme'] + '.pdf', 'F')

In [ ]:
df_corpus_langue=[] 
#ouverture de chaque onglet Corpus de travail dans un DF
for index, item in enumerate(vk_lang_dict):
  df_corpus_langue.append (charge_corpus_langue(df_corpus, item['trigramme']))
  create_pdf_instance(item, df_corpus_langue[index])


In [ ]:
ls -l /usr/share/fonts/truetype/dejavu/

total 2956
-rw-r--r-- 1 root root 705684 Jul 30  2016 DejaVuSans-Bold.ttf
-rw-r--r-- 1 root root    549 Jun 18 14:47 DejaVuSans.cw127.pkl
-rw-r--r-- 1 root root 331992 Jul 30  2016 DejaVuSansMono-Bold.ttf
-rw-r--r-- 1 root root 340712 Jul 30  2016 DejaVuSansMono.ttf
-rw-r--r-- 1 root root 136927 Jun 18 14:47 DejaVuSans.pkl
-rw-r--r-- 1 root root 757076 Jul 30  2016 DejaVuSans.ttf
-rw-r--r-- 1 root root 356088 Jul 30  2016 DejaVuSerif-Bold.ttf
-rw-r--r-- 1 root root 380132 Jul 30  2016 DejaVuSerif.ttf
